In [7]:
import os

In [8]:
import os
os.chdir(r'C:\Users\YASWANTH\Desktop\Text summary project\Text-Summary-project-ongoing')  # Change directory
print(os.getcwd())  # Print working directory


C:\Users\YASWANTH\Desktop\Text summary project\Text-Summary-project-ongoing


In [9]:
pwd

'C:\\Users\\YASWANTH\\Desktop\\Text summary project\\Text-Summary-project-ongoing'

In [10]:
%pip install transformers accelerate

Note: you may need to restart the kernel to use updated packages.


In [8]:
%pip install transformers 


Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install transformers[torch]

Note: you may need to restart the kernel to use updated packages.


In [11]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int

In [12]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [14]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_ckpt = config.model_ckpt,
            num_train_epochs = params.num_train_epochs,
            warmup_steps = params.warmup_steps,
            per_device_train_batch_size = params.per_device_train_batch_size,
            weight_decay = params.weight_decay,
            logging_steps = params.logging_steps,
            evaluation_strategy = params.evaluation_strategy,
            eval_steps = params.evaluation_strategy,
            save_steps = params.save_steps,
            gradient_accumulation_steps = params.gradient_accumulation_steps
        )

        return model_trainer_config

In [15]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
import torch

c:\Users\YASWANTH\anaconda3\envs\texts\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config


    
    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)
        
        #loading data 
        dataset_samsum_pt = load_from_disk(self.config.data_path)

        # trainer_args = TrainingArguments(
        #     output_dir=self.config.root_dir, num_train_epochs=self.config.num_train_epochs, warmup_steps=self.config.warmup_steps,
        #     per_device_train_batch_size=self.config.per_device_train_batch_size, per_device_eval_batch_size=self.config.per_device_train_batch_size,
        #     weight_decay=self.config.weight_decay, logging_steps=self.config.logging_steps,
        #     evaluation_strategy=self.config.evaluation_strategy, eval_steps=self.config.eval_steps, save_steps=1e6,
        #     gradient_accumulation_steps=self.config.gradient_accumulation_steps
        # ) 


        trainer_args = TrainingArguments(
            output_dir=self.config.root_dir, num_train_epochs=1, warmup_steps=100,
            per_device_train_batch_size=1, per_device_eval_batch_size=1,
            weight_decay=0.01, logging_steps=10,
            evaluation_strategy='steps', eval_steps=100, save_steps=1e6,
            gradient_accumulation_steps=16
        ) 

        trainer = Trainer(model=model_pegasus, args=trainer_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=dataset_samsum_pt["test"].select(range(10)), 
                  eval_dataset=dataset_samsum_pt["validation"].select(range(10))
        )
        
        trainer.train()
        

        ## Save model
        model_pegasus.save_pretrained(os.path.join(self.config.root_dir,"pegasus-samsum-model"))
        ## Save tokenizer
        tokenizer.save_pretrained(os.path.join(self.config.root_dir,"tokenizer"))

In [17]:
import os
os.chdir(r'C:\Users\YASWANTH\Desktop\Text summary project\Text-Summary-project-ongoing')  # Change directory
print(os.getcwd())  # Print working directory


C:\Users\YASWANTH\Desktop\Text summary project\Text-Summary-project-ongoing


In [16]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2025-02-17 01:28:56,130: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-02-17 01:28:56,138: INFO: common: yaml file: params.yaml loaded successfully]
[2025-02-17 01:28:56,140: INFO: common: created directory at: C:\Users\YASWANTH\Desktop\Text summary project\Text-Summary-project-ongoing/artifacts]
[2025-02-17 01:28:56,146: INFO: common: created directory at: C:\Users\YASWANTH\Desktop\Text summary project\Text-Summary-project-ongoing/artifacts/model_trainer]


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\YASWANTH\anaconda3\envs\texts\lib\site-packages\transformers\training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\YASWANTH\AppData\Local\Temp\ipykernel_4528\1512205139.py:33: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model=model_pegasus, args=trainer_args,
100%|██████████| 1/1 [07:51<00:00, 471.91s/it]c:\Users\YASWANTH\anaconda3\envs\texts\lib\site-packages\transformers\modeling_utils.py:2817: UserWa

{'train_runtime': 596.342, 'train_samples_per_second': 0.017, 'train_steps_per_second': 0.002, 'train_loss': 2.100613832473755, 'epoch': 1.0}
